# Mapa interativo para busca de clientes no estado de Pernambuco

## Bibliotecas utilizadas no projeto


In [1]:
import pandas as pd
import geopandas as gd
import keplergl as kp

## Limpeza e tratamento dos dados da receita federal

O volume de dados fornecido pela Receita Federal é elevado, porém segue um único padrão que facilitará o processo como um todo. Tendo em vista este padrão, realizaremos o tratamento de um dos conjuntos de dados e construiremos uma função para tratamento dos demais conjuntos de dados.

OBS: temos algumas informações relevantes sobre o arquivo que são necessárias ao correto carregamento dos dados no pandas, são elas:
- O arquivo é do tipo CSV, logo, utilizaremos a função read_csv do pandas;
- O separador de colunas deste arquivo é ";" e indicaremos isto com o argumento "sep='; "';
- O arquivo foi construído com unicode ANSI e indicaremos isto com o argumento "encoding='ANSI";
- O arquivo não possui títulos para as colunas e precisamos especificar esta informação pare evitar a "promoção" da primeira linha como título da coluna, para isto utilizaremos o argumento "header=None";
- Passaremos a lista de titulos "titulos_colum" para o argumento "names=" que fará a atribuição dos títulos ao dataframe.

Dadas as observações criamos duas variáveis para armazenar o caminho do arquivo e títulos das colunas 


In [4]:
#caminho do arquivo
caminho_dados = "C:/Users/humberto.araujo.ext/OneDrive - Cimento Nacional/ATIVIDADES/2025-06-23_geoint/ENTRADAS/estab_01.csv"

#Nome das colunas para reatribuição
titulos_colum = [
    "cnpj_basico",
    "cnpj_ordem",
    "cnjp_dv",
    "identificador_matriz",
    "nome_fantasia",
    "situacao_cadastral",
    "data_situacao",
    "motivo_situacao",
    "nome_cidade_exterior",
    "pais",
    "data_inicio_atv",
    "CNAE_principal",
    "CNAE_secundario",
    "tipo_logradouro",
    "logradouro",
    "numero_logra",
    "complemento",
    "bairro",
    "CEP",
    "UF",
    "municipio",
    "ddd1","telefone1",
    "ddd2","telefone2",
    "ddd_fax",'fax',
    "email",
    "situacao_especial",
    "data_situacao_especial"
]

Após a construção das variáveis de apoio realizaremos a carga dos dados com seus devidos cuidados(separador, codificação,título).

In [ ]:
#carga dos dados
df_RF_01 = pd.read_csv(caminho_dados,sep=';',encoding='ANSI', header=None,names=titulos_colum)

C:\Users\humberto.araujo.ext\AppData\Local\Temp\ipykernel_47888\3134606272.py:2: DtypeWarning: Columns (8,21,22,24,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_RF_01 = pd.read_csv(caminho_dados,sep=';',encoding='ANSI', header=None,names=titulos_colum)


,cnpj_basico,cnpj_ordem,cnjp_dv,identificador_matriz,nome_fantasia,situacao_cadastral,data_situacao,motivo_situacao,nome_cidade_exterior,pais,...,municipio,ddd1,telefone1,ddd2,telefone2,ddd_fax,fax,email,situacao_especial,data_situacao_especial
0,7396865,1,68,1,NaN,8,20170210,1,NaN,NaN,...,8297,47.0,33851125,47.0,33851125.0,47.0,33851125,NaN,NaN,NaN
1,64904295,18,51,2,NaN,8,20161110,1,NaN,NaN,...,2785,11.0,36491000,31.0,33880436.0,82.0,33118379,CLAUDIO.GIGLIO@CAMIL.COM.BR,NaN,NaN


Nosso objetivo é listar apenas as empresas em situação ativa, neste sentido raelizaremos um filtro na coluna "situacao_cadastral" para todos os correspondentes ao valor "2" que conforme dicionário de dados corresponde as empresas ativas.

In [18]:
#filtro de cnpj's ativos
df_RF_ativos = df_RF_01[df_RF_01['situacao_cadastral']==2 ]
df_RF_ativos.head(2)

,cnpj_basico,cnpj_ordem,cnjp_dv,identificador_matriz,nome_fantasia,situacao_cadastral,data_situacao,motivo_situacao,nome_cidade_exterior,pais,...,municipio,ddd1,telefone1,ddd2,telefone2,ddd_fax,fax,email,situacao_especial,data_situacao_especial
12,591451,1,40,1,QUIM-BASE,2,20051103,0,NaN,NaN,...,7107,11.0,56415044,NaN,NaN,11.0,56415044,UTILEX.LEGAL@BOL.COM.BR,NaN,NaN
16,7396980,1,32,1,NaN,2,0,0,NaN,NaN,...,6555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Faremos a remoção das colunas não interessantes ao nosso objetivo

In [ ]:
#drop de colunas
df_RF_ativos = df_RF_ativos.drop(['situacao_cadastral','data_situacao','motivo_situacao','nome_cidade_exterior','pais','situacao_especial','data_situacao_especial'], axis=1)

KeyError: "['situacao_cadastral', 'data_situacao', 'motivo_situacao', 'nome_cidade_exterior', 'pais', 'situacao_especial', 'data_situacao_especial'] not found in axis"

Por fim, faremos um filtro de estado e salvaremos o resultado em um arquivo do tipo csv

In [23]:
#Filtro para coletar dados do estado de pernambuco
df_RF_PE = df_RF_ativos[df_RF_ativos['UF']=='PE']

#Exportando os valores salvos em um arquivo do tipo csv
df_RF_PE.to_csv('Dados_pe/pe_01')

### Funçao para limpeza dos dados

Como visto o processo de limepeza ocorreu conforme o desejado e neste ponto criaremos uma função que realize o processamento dos demais arquivos

In [ ]:
def pre_limpeza(lista_arquivos ,destino_salvamento):

    for i in lista_arquivos:
    